# Prerequisites
- Python 3
- Python jupyter module
- Python clearml module
- [Cuda 12.3](https://developer.nvidia.com/cuda-downloads)

# Set ENV vars

In [2]:
%env CLEARML_WEB_HOST=http://clearml.ad.ai.sdl.com:8080
%env CLEARML_API_HOST=http://clearml.ad.ai.sdl.com:8008
%env CLEARML_FILES_HOST=http://clearml.ad.ai.sdl.com:8081
%env CLEARML_API_ACCESS_KEY=SKXA1SPMCLDW7NH3JG5F
%env CLEARML_API_SECRET_KEY=bMKqfrJnZOWTIJEgnYVfBSncCub4XBPxGYhspwitWFmREfYKUe

env: CLEARML_WEB_HOST=http://clearml.ad.ai.sdl.com:8080
env: CLEARML_API_HOST=http://clearml.ad.ai.sdl.com:8008
env: CLEARML_FILES_HOST=http://clearml.ad.ai.sdl.com:8081
env: CLEARML_API_ACCESS_KEY=SKXA1SPMCLDW7NH3JG5F
env: CLEARML_API_SECRET_KEY=bMKqfrJnZOWTIJEgnYVfBSncCub4XBPxGYhspwitWFmREfYKUe


# Create dataset

In [2]:
from clearml import Dataset
dataset = Dataset.create(
  dataset_name='kaggle_sarcasm',
  dataset_project='sarcasm_detector',
  dataset_version="1.0",
  description='From https://www.kaggle.com/datasets/danofer/sarcasm'
)

ClearML results page: http://clearml.ad.ai.sdl.com:8080/projects/8d45228c5d8e47c0be6c81baf44c0eea/experiments/fd9a1a524e4149ecbd29cae28c7888f6/output/log
ClearML dataset page: http://clearml.ad.ai.sdl.com:8080/datasets/simple/8d45228c5d8e47c0be6c81baf44c0eea/experiments/fd9a1a524e4149ecbd29cae28c7888f6


# Upload files

In [5]:
dataset.add_files("data")
dataset.upload()
dataset.finalize()

Generating SHA2 hash for 2 files


100%|██████████| 2/2 [00:00<00:00,  8.86it/s]


Hash generation completed
Uploading dataset changes (2 files compressed to 28.69 MiB) to http://clearml.ad.ai.sdl.com:8081
File compression and upload completed: total size 28.69 MiB, 1 chunk(s) stored (average size 28.69 MiB)


True

# Start sklearn training

In [ ]:
from train_sklearn import SklearnTrainer

sarcasm_trainer = SklearnTrainer(subset_size=1000)
sarcasm_trainer.train()
sarcasm_trainer.task.close()

Retrying (Retry(total=239, connect=239, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D3D269BD90>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed')': /auth.login
Retrying (Retry(total=238, connect=238, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D3D32F1410>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed')': /auth.login
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D3D3253210>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed')': /auth.login
Retrying (Retry(total=236, connect=236, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object

Check pytorch versions

In [4]:
import torch

print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.1.0+cpu
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Sep__8_19:56:38_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.3, V12.3.52
Build cuda_12.3.r12.3/compiler.33281558_0
**********
CUDNN version: None
Available GPU devices: 0


AssertionError: Torch not compiled with CUDA enabled

# Train transformer

In [3]:
from clearml import Task
from train_transformer import SarcasmTrainer

Task.add_requirements("torch")
Task.init(project_name="sarcasm_detector", task_name="DistilBert Training")
sarcasm_trainer = SarcasmTrainer(subset_size=1000)
sarcasm_trainer.train()
Task.close()

ClearML Task: created new task id=7dcc42c34c7a4b8ca40d501d65c03668
ClearML results page: http://clearml.ad.ai.sdl.com:8080/projects/2bb1d55901b4402d80b7120beadfc8dc/experiments/7dcc42c34c7a4b8ca40d501d65c03668/output/log
2023-10-24 21:47:24,649 - clearml.Task - INFO - Storing jupyter notebook directly as code


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using custom data configuration default-4fa3bd361ffaa254
Found cached dataset csv (file://C:/Users/Andrew/.cache/huggingface/datasets/csv/default-4fa3bd361ffaa254/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.